In [2]:
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
ps = PorterStemmer()

In [6]:
def all_the_cooking(line):
    review = re.sub('[^a-zA-z]' , ' ', line)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    return(review)

In [7]:
for i,line in enumerate(data.tweet):
    data.tweet[i] = all_the_cooking(line)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
data.head()

,id,label,tweet
0,1,0,user father dysfunct selfish drag kid dysfunct...
1,2,0,user user thank lyft credit use caus offer whe...
2,3,0,bihday majesti
3,4,0,model love u take u time ur
4,5,0,factsguid societi motiv


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.5, stratify=data['label'])

trainp=data[data.label==1]
trainn=data[data.label==0]
print(trainp.info())
trainn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2242 entries, 13 to 31960
Data columns (total 3 columns):
id       2242 non-null int64
label    2242 non-null int64
tweet    2242 non-null object
dtypes: int64(2), object(1)
memory usage: 70.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29720 entries, 0 to 31961
Data columns (total 3 columns):
id       29720 non-null int64
label    29720 non-null int64
tweet    29720 non-null object
dtypes: int64(2), object(1)
memory usage: 928.8+ KB


In [11]:
from imblearn.over_sampling import SMOTE

In [12]:
sm = SMOTE(random_state = 2)

In [14]:
vect = TfidfVectorizer()
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [15]:
tf_xtrain_sm, tf_ytrain_sm = sm.fit_sample(tf_train, y_train)

In [17]:
print(tf_train.shape)
tf_xtrain_sm.shape

(15981, 20133)


(29720, 20133)

In [18]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X=tf_train, y = y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
expected = y_test
predicted=model.predict(tf_test)

In [20]:
from sklearn import metrics

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97     14860
           1       1.00      0.12      0.21      1121

    accuracy                           0.94     15981
   macro avg       0.97      0.56      0.59     15981
weighted avg       0.94      0.94      0.91     15981

[[14860     0]
 [  987   134]]


In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predicted)

In [22]:
print(accuracy)

0.938239159001314


In [24]:
metrics.f1_score(y_test,predicted)

0.21354581673306772